<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/HAFS/hafs_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install botocore
!pip install xarray
!pip install matplotlib
!pip install xmovie

!apt-get update && apt-get install -y ffmpeg netcdf4
!pip install netcdf4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import boto3
import botocore

from datetime import datetime, timedelta
import xarray as xr
from xmovie import Movie

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def download_s3_file(BUCKET_NAME, KEY, fname):
    """
    Downloads a file from an AWS S3 bucket.

    Args:
        BUCKET_NAME (str): The name of the S3 bucket.
        KEY (str): The key (prefix) of the object in the bucket.
        fname (str): The filename of the object to download.

    Returns:
        str or None: The local filename if successful, None otherwise.
    """
    # KEY_WITH_FNAME = f'{KEY}{fname}' # Construct the full key with the filename - This was incorrect

    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))

    try:
        s3.download_file(BUCKET_NAME, f'{KEY}/{fname}', fname) # Use the constructed key and fname for local filename
        print(f"Successfully downloaded {KEY}/{fname} to {fname}")
        return fname
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print(f"The object does not exist: {KEY}/{fname}")
        else:
            print(f"An error occurred: {e}")
        return None

# Inputs

In [ ]:
BUCKET_NAME = 'noaa-nws-hafs-pds'

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/'

date_start, date_end = ['20251023', '20251031'] # before 10/22 Melissa was Invest 98L
forecast_hour = '00' # Out of forecasts at 4-synoptic times, only `00`.
hr_start, hr_end, freq = [0, 97, 3]

basin = "13l"
which_hafs = "hfsa"
model = "mom6"

vars_to_save = ['temp', 'so']

# domain indices
x_start, x_end = [-100, -40] # longitude
y_start, y_end = [10, 45] # latitude
z_start, z_end = [0, 1001] # depth (m) to include 1000 m

# Jamaica? In x- and y- indices! The nc file is POORLY constructed.
x_pos = 310
y_pos_start, y_pos_end = [200, 250]

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/' + which_hafs.upper() + '/'

## Get the data

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

for forecast_start_date in date_list:
  dStr = forecast_start_date.strftime('%Y%m%d')
  KEY = f'{which_hafs}/{dStr}/{forecast_hour}'
  print(f'KEY: {KEY}')
  filenames = [f'{basin}.{dStr}{forecast_hour}.{which_hafs}.{model}.f{hour:03d}.nc' for hour in range(hr_start, hr_end, freq)]

  for fname in filenames:
    print(f'Downloading: {fname}')
    download_s3_file(BUCKET_NAME, KEY, fname)
    ds = xr.open_dataset(fname)
    ds_cull = ds[vars_to_save]
    ds_subset = ds_cull.sel(xh=slice(x_start, x_end), yh=slice(y_start, y_end), z_l=slice(z_start, z_end))
    fname_subset = fname.replace('.nc', '_subset.nc')
    ds_subset.to_netcdf(fname_subset)
    !rm {fname}

  subset_filenames = [fname.replace('.nc', '_subset.nc') for fname in filenames]
  ds = xr.open_mfdataset(subset_filenames)
  fName_save = f'{save_data_path}' + dStr + '.nc'
  print(f'Saving to {fName_save}')
  ds.to_netcdf(fName_save)

  for fname in subset_filenames:
    !rm {fname}

## Animate

In [ ]:
#movie = Movie(ds.isel(z_l=0).temp,
#              framedim='time',
#              input_check=False,
#              x='xh',
#              y='yh',
#              vmin=25,
#              vmax=32,
#              cmap='Spectral_r')
#movie.save('temperature_animation_xmovie.gif')

#movie = Movie(ds.isel(z_l=0).so,
#              framedim='time',
#              input_check=False,
#              x='xh',
#              y='yh',
#              vmin=34,
#              vmax=38,
#              cmap='jet')
#movie.save('salinity_animation_xmovie.gif')